In [66]:
import pandas as pd
import matplotlib.pylab as plt
import re
from sklearn.feature_extraction import text
from spellchecker import SpellChecker

## Data preprocess

In [69]:
# Read csv
hscode_df = pd.read_csv("./data/fta_hscode.csv", encoding="utf-8",
                       dtype={'HS_CODE': str, 'ITEM_NAME': str})
hscode_df.HS_CODE = hscode_df.HS_CODE.map(lambda x: '0' + x if len(x) < 10 else x)
hscode_df.ITEM_NAME = hscode_df.ITEM_NAME.str.lower()

In [24]:
# Slice code
def code_slice(df, column, n):
    print("Max Length of Original Code: ", hscode_df[column].map(len).max())
    print("Min Length of Original Code: ", hscode_df[column].map(len).min())
    df.HS_CODE = hscode_df[column].str.slice(stop=n)
    print("Sliced Length of Code: ", n)
    return df

In [31]:
def delete_blank_rows(df, column):
    origin_length = len(df)
    print("Number of Rows in Dataframe (Before): ", origin_length)
    df.dropna(subset=[column], inplace=True)
    df[column] = df[column].str.strip()
    df = df[df[column].map(lambda x: x!='')]
    print("Number of Deleted Rows in Dataframe: ", origin_length - len(df))
    print("Number of Rows in Dataframe (After): ", len(df))
    return df

In [73]:
def process_special_char(df, column):
    df[column] = df[column]\
        .map(lambda x: ' '.join(re.sub(r"[^a-z0-9 ]", " ", word).strip() for word in str(x).split()))
    return df

In [ ]:
def 

In [50]:
# Check Special Char
hscode_df.loc[hscode_df.ITEM_NAME.str.contains(r"[^A-Za-z0-9 \-]")]

,HS_CODE,ITEM_NAME
5,8419909090,part(air preheater)
36,0202201000,"frozen beef rib, back rib bone in"
44,0202201000,"frozen beef rib, short rib"
45,0202201000,"frozen beef rib,short rib b-in select or higher"
51,0202201000,"frozen beef rib, short rib bone in usda se/hi"
53,0202201000,"frozen beef rib, short rib bone in usda select or"
56,0202201000,"frozen beef rib,short rib tips bone-in"
57,0202201000,"frozen beefrib, short rib tips bone-in"
61,0202201000,"frozen beef short rib,back rib"
68,0202201000,frozen beef s srb 3 ribs mw/vac


In [68]:
re.sub(r"[^a-z0-9 ]", " ", "rubbing？strip,？rubbing？vrs2833")

'rubbing strip  rubbing vrs2833'

In [32]:
hscode_df = delete_blank_rows(hscode_df, 'ITEM_NAME')
hscode_df.head()

Number of Rows in Dataframe (Before):  2071580
Number of Deleted Rows in Dataframe:  0
Number of Rows in Dataframe (After):  2071580


,HS_CODE,ITEM_NAME
0,8419909090,PAPER MACHINE PARTS
1,8419909090,PARAFFIN DISSOLVER PARTS
2,8419909090,PARA THIMMER
3,8419909090,PARS FOR CONSTANT TEMPERATURE AND HUMIDITY CHA...
4,8419909090,PART


In [ ]:
hscode_df.loc[hscode_df.ITEM_NAME == 'WELL'].drop_duplicates('HS_CODE')

In [7]:
# Delete stopwords in ITEM_NAME
stop_words = [word.upper() for word in text.ENGLISH_STOP_WORDS]
hscode_df['ITEM_NAME_WITHOUT_STOPWORDS'] = hscode_df.ITEM_NAME.map(lambda x: ' '.join(word for word in str(x).upper().split() if word not in (stop_words)))
hscode_df.head()

,HS_CODE,ITEM_NAME,ITEM_NAME_WITHOUT_STOPWORDS
0,8419909090,PAPER MACHINE PARTS,PAPER MACHINE PARTS
1,8419909090,PARAFFIN DISSOLVER PARTS,PARAFFIN DISSOLVER PARTS
2,8419909090,PARA THIMMER,PARA THIMMER
3,8419909090,PARS FOR CONSTANT TEMPERATURE AND HUMIDITY CHA...,PARS CONSTANT TEMPERATURE HUMIDITY CHAMBER
4,8419909090,PART,


In [ ]:
hscode_df.loc[hscode_df.ITEM_NAME.fillna('NULL').str.contains('FOR')]

In [ ]:
# Hscode count
hscode_count = hscode_df.groupby('HS_CODE').agg('count').rename(columns={'ITEM_NAME':'COUNT'}).sort_values('COUNT', ascending=False)
hscode_count.head()

In [ ]:
hscode_count.sort_values('COUNT').plot.box()

In [ ]:
hscode_df.loc[hscode_df.HS_CODE == '392690'].drop_duplicates('ITEM_NAME')

In [ ]:
name_count = hscode_df.groupby('ITEM_NAME').agg('count').rename(columns={'HS_CODE':'COUNT'}).sort_values('COUNT', ascending=False)
name_count.head()

In [ ]:
len(name_count)

In [ ]:
name_count.plot.box()

In [ ]:
name_count.describe()

In [ ]:
hscode_df.groupby('HS_CODE').agg('count').plot.box()

In [ ]:
hscode_df.groupby('HS_CODE').agg('count').sort_values('ITEM_NAME', ascending=False)

In [ ]:
hscode_df.groupby('HS_CODE').agg('count').describe()

In [ ]:
df3 = hscode_df.groupby('HS_CODE').agg('count')

In [ ]:
iqr = df3.quantile(0.75) - df3.quantile(0.25)
upper_bound = df3.quantile(0.75) + 1.5*iqr
lower_bound = df3.quantile(0.25) - 1.5*iqr
print(lower_bound)
df3 = df3[(df3 < upper_bound) & (df3 > lower_bound) ]

In [ ]:
df3.plot.box()

In [ ]:
df3.describe()

In [ ]:
hscode_df.groupby('HS_CODE').agg('count').sort_values('ITEM_NAME', ascending=False)[1100:].plot.box()

In [ ]:
name_count.plot()

In [ ]:
name_count[3:750000].plot()

In [ ]:
len(name_count)

In [ ]:
name_count.iloc[100000]

In [ ]:
hscode_df.loc[hscode_df.ITEM_NAME == 'BUTEETRAPPECARBURANT']

In [ ]:
name_code_group = hscode_df.groupby(['ITEM_NAME', 'HS_CODE']).agg('count').reset_index()
name_code_group.head()

In [ ]:
name_code_group[name_code_group.ITEM_NAME.duplicated(keep=False)]

In [ ]:
name_code_group[name_code_group.ITEM_NAME.duplicated(keep=False)].drop_duplicates('ITEM_NAME')

In [ ]:
name_code_group[name_code_group.ITEM_NAME.duplicated(keep=False)].iloc[582422:-13].drop_duplicates('ITEM_NAME')

In [ ]:
name_code_group[~name_code_group.ITEM_NAME.duplicated(keep=False)]

In [ ]:
hscode_df.ITEM_NAME.str.upper().str.replace(r"[^A-Z ]", "", regex=True)

In [ ]:
re_hscode_df = hscode_df
re_hscode_df.ITEM_NAME = re_hscode_df.ITEM_NAME.str.upper().str.replace(r"[^A-Z ]", "", regex=True)
re_hscode_df.head()

In [ ]:
re_hscode_df.dropna(inplace=True)
re_hscode_df.loc[re_hscode_df.ITEM_NAME.isna()]

In [ ]:
re_hscode_df.ITEM_NAME = re_hscode_df.ITEM_NAME.str.strip()

In [ ]:
re_hscode_df.sort_values('ITEM_NAME').head()

In [ ]:
re_hscode_df.drop(re_hscode_df[re_hscode_df.ITEM_NAME == '   '].index, inplace=True)
re_hscode_df.loc[re_hscode_df.ITEM_NAME == '']

In [ ]:
re_hscode_df.groupby('HS_CODE').agg('count').sort_values('ITEM_NAME').plot()

In [ ]:
re_hscode_df.groupby('HS_CODE').agg('count').sort_values('ITEM_NAME', ascending=False).head()

In [ ]:
re_hscode_df.groupby(['HS_CODE', 'ITEM_NAME']).agg('count').reset_index()

In [ ]:
re_hscode_df_name_count = re_hscode_df.groupby('ITEM_NAME').agg('count')\
                        .rename(columns={'HS_CODE': 'COUNT'}).reset_index()\
                        .sort_values('COUNT', ascending=False)
re_hscode_df_name_count.head()

In [ ]:
re_hscode_df_name_count[1:100].plot(x='ITEM_NAME', y="COUNT")

In [ ]:
# need to fix
re_hscode_df.groupby(['HS_CODE', 'ITEM_NAME']).agg('count').reset_index()

In [ ]:
re_hscode_df.drop_duplicates(inplace=True)
re_hscode_df.head()

In [ ]:
# Check above
re_hscode_df.loc[re_hscode_df.ITEM_NAME == 'PART'].sort_values('HS_CODE').head()

In [ ]:
name_count[~name_count.index.str.upper().str.contains(r"[^A-Z ]")]

In [ ]:
len(name_count)

In [ ]:
test = hscode_df.ITEM_NAME.str.upper().str.contains(r"[^A-Z ]")

In [ ]:
test.loc[(test != False) & (test != True)]

In [ ]:
off_special_char = hscode_df[~test.fillna(True)]
off_special_char.head()

In [ ]:
off_special_char.HS_CODE = off_special_char.HS_CODE.str.slice(stop=6)
off_special_char.head()

In [ ]:
off_special_char.ITEM_NAME = off_special_char.ITEM_NAME.str.upper()

In [ ]:
off_special_char.sort_values('ITEM_NAME')

### Make train, test set

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(re_hscode_df, test_size=0.2)

In [ ]:
train.ITEM_NAME.to_csv('train_name_fta_spcial_char_off.txt', index=False, header=False)

In [ ]:
train.HS_CODE.to_csv('train_class_fta_spcial_char_off.txt', index=False, header=False)

In [ ]:
test.ITEM_NAME.to_csv('test_name_fta_spcial_char_off.txt', index=False, header=False)

In [ ]:
test.HS_CODE.to_csv('test_class_fta_spcial_char_off.txt', index=False, header=False)

In [ ]:
train.sort_values('ITEM_NAME')

In [ ]:
train, test = train_test_split(re_hscode_df, test_size=0.2)

In [ ]:
train.ITEM_NAME.to_csv('train_name_fta.txt', index=False, header=False)
train.HS_CODE.to_csv('train_class_fta.txt', index=False, header=False)
test.ITEM_NAME.to_csv('test_name_fta.txt', index=False, header=False)
test.HS_CODE.to_csv('test_class_fta.txt', index=False, header=False)

In [ ]:
train.ITEM_NAME.to_csv('train_name_fta_alpha_word.txt', index=False, header=False)
train.HS_CODE.to_csv('train_class_fta_alpha_word.txt', index=False, header=False)
test.ITEM_NAME.to_csv('test_name_fta_alpha_word.txt', index=False, header=False)
test.HS_CODE.to_csv('test_class_fta_alpha_word.txt', index=False, header=False)

## Make Unknown Words Dictionary

In [74]:
test_df = process_special_char(hscode_df, 'ITEM_NAME')

In [96]:
a = [i.split() for i in test_df.ITEM_NAME]
# b = [j.split() for j in a]

In [98]:
a

[['paper', 'machine', 'parts'],
 ['paraffin', 'dissolver', 'parts'],
 ['para', 'thimmer'],
 ['pars', 'for', 'constant', 'temperature', 'and', 'humidity', 'chamber'],
 ['part'],
 ['part', 'air', 'preheater'],
 ['part', 'cleaner'],
 ['part', 'cleaner', 'and', 'parts'],
 ['part', 'coffee', 'roaster'],
 ['part', 'for'],
 ['part', 'for', '7000', 'pph', 'modular', 'ccr', 'regen', 'section'],
 ['part', 'for', 'air', 'cleaner'],
 ['part', 'for', 'air', 'conditioner'],
 ['part', 'for', 'airconditioner'],
 ['part', 'for', 'airconditioning', 'machine'],
 ['part', 'for', 'airconditioning', 'machine'],
 ['part', 'for', 'air', 'cooled', 'heat', 'exchanger'],
 ['part', 'for', 'air', 'cooler'],
 ['part', 'for', 'aircraft'],
 ['frozen', 'beef', 'cow', 'short', 'rib'],
 ['frozen', 'beef', 'cow', 'short', 'rib', '3', '7', 'iw'],
 ['frozen', 'beef', 'c', 'short', 'rib', '7', '11'],
 ['frozen', 'beef', 'c', 'shortribs', '2', '6r', 'iw'],
 ['frozen', 'beef', 'economy', 'back', 'rib'],
 ['frozen', 'beef', 'o